In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from transformers import BertTokenizer, BertModel
import Levenshtein as lev
import os
import pandas as pd
import re
import yaml
import torch


c:\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_yaml_data(folder_path):
    constructs_list = []
    hypotheses_data = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.yaml') or file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r') as file:
                yaml_content = yaml.safe_load(file)

                # Extract constructs
                constructs = yaml_content.get('constructs', {})
                constructs_list.extend(constructs.values())

                # Extract hypotheses
                for hypothesis in yaml_content.get('hypotheses', {}).values():
                    cause = constructs.get(hypothesis['cause'], '')
                    effect = constructs.get(hypothesis['effect'], '')
                    hypotheses_data.append({'cause': cause, 'effect': effect})

    # Remove duplicates from constructs list
    constructs_list = list(set(constructs_list))

    # Create DataFrame for hypotheses
    hypotheses_df = pd.DataFrame(hypotheses_data)

    return constructs_list, hypotheses_df

In [3]:
# Load the constructs and hypotheses
constructs_true, hypotheses_true = load_yaml_data('../true_results')
constructs_detected, hypotheses_detected = load_yaml_data('../chatGPT_fewshot5_yaml')

In [7]:
def is_similar(str1, str2, max_distance=1):
    return lev.distance(str1, str2) <= max_distance

def map_constructs(constructs_true, constructs_detected):
    mapped_constructs = []
    for detected in constructs_detected:
        similar_constructs = [tru for tru in constructs_true if is_similar(detected, tru, max_distance=0)]
        if similar_constructs:
            mapped_constructs.append((detected, similar_constructs))
    return mapped_constructs

mapped_constructs = map_constructs(constructs_true, constructs_detected)
print(len(mapped_constructs))
for detected, true_list in mapped_constructs:
    print(f"Detected Construct: {detected}")
    print(f"True Constructs:    {true_list}")

179
Detected Construct: Fairness Effect
True Constructs:    ['Fairness Effect']
Detected Construct: Staff productivity increased
True Constructs:    ['Staff productivity increased']
Detected Construct: Outstanding credibility text comments
True Constructs:    ['Outstanding credibility text comments']
Detected Construct: Leadership
True Constructs:    ['Leadership']
Detected Construct: System Quality
True Constructs:    ['System Quality']
Detected Construct: Technology Functionality
True Constructs:    ['Technology Functionality']
Detected Construct: Behavioral Intention to Use
True Constructs:    ['Behavioral Intention to Use']
Detected Construct: Mobile Privacy Concern
True Constructs:    ['Mobile Privacy Concern']
Detected Construct: Nearby suppliers
True Constructs:    ['Nearby suppliers']
Detected Construct: IT work experience
True Constructs:    ['IT work experience']
Detected Construct: Using the Prescribed Tools and Techniques
True Constructs:    ['Using the Prescribed Tools and